<a href="https://colab.research.google.com/github/michelmalki/MichelMalki/blob/main/CCCS_680_Assignment_3_Question_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [115]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("CCCS 680 Assignment 3 Question 2") \
       .getOrCreate()

spark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,626 B in 1s (2,614 B/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
46 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire 

In [116]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [117]:
visitors_df = spark.read.csv('/content/drive/My Drive/CCCS-680 Assignment 3/White_House_Visitor_Records_Requests.csv.gz', inferSchema=True, header=True, sep=',')
visitors_df

DataFrame[NAMELAST: string, NAMEFIRST: string, NAMEMID: string, UIN: string, BDGNBR: int, Type of Access: string, TOA: string, POA: string, TOD: string, POD: string, APPT_MADE_DATE: string, APPT_START_DATE: string, APPT_END_DATE: string, APPT_CANCEL_DATE: string, Total_People: int, LAST_UPDATEDBY: string, POST: string, LastEntryDate: string, TERMINAL_SUFFIX: string, visitee_namelast: string, visitee_namefirst: string, MEETING_LOC: string, MEETING_ROOM: string, CALLER_NAME_LAST: string, CALLER_NAME_FIRST: string, CALLER_ROOM: string, Description: string, RELEASE_DATE: string]

In [118]:
visitors_df.dtypes

[('NAMELAST', 'string'),
 ('NAMEFIRST', 'string'),
 ('NAMEMID', 'string'),
 ('UIN', 'string'),
 ('BDGNBR', 'int'),
 ('Type of Access', 'string'),
 ('TOA', 'string'),
 ('POA', 'string'),
 ('TOD', 'string'),
 ('POD', 'string'),
 ('APPT_MADE_DATE', 'string'),
 ('APPT_START_DATE', 'string'),
 ('APPT_END_DATE', 'string'),
 ('APPT_CANCEL_DATE', 'string'),
 ('Total_People', 'int'),
 ('LAST_UPDATEDBY', 'string'),
 ('POST', 'string'),
 ('LastEntryDate', 'string'),
 ('TERMINAL_SUFFIX', 'string'),
 ('visitee_namelast', 'string'),
 ('visitee_namefirst', 'string'),
 ('MEETING_LOC', 'string'),
 ('MEETING_ROOM', 'string'),
 ('CALLER_NAME_LAST', 'string'),
 ('CALLER_NAME_FIRST', 'string'),
 ('CALLER_ROOM', 'string'),
 ('Description', 'string'),
 ('RELEASE_DATE', 'string')]

In [119]:
visitors_df.show()

+------------+---------+-------+----+------+--------------+----+----+----+----+--------------+---------------+-------------+----------------+------------+--------------+----+-------------+---------------+----------------+-----------------+-----------+------------+----------------+-----------------+-----------+-----------+--------------------+
|    NAMELAST|NAMEFIRST|NAMEMID| UIN|BDGNBR|Type of Access| TOA| POA| TOD| POD|APPT_MADE_DATE|APPT_START_DATE|APPT_END_DATE|APPT_CANCEL_DATE|Total_People|LAST_UPDATEDBY|POST|LastEntryDate|TERMINAL_SUFFIX|visitee_namelast|visitee_namefirst|MEETING_LOC|MEETING_ROOM|CALLER_NAME_LAST|CALLER_NAME_FIRST|CALLER_ROOM|Description|        RELEASE_DATE|
+------------+---------+-------+----+------+--------------+----+----+----+----+--------------+---------------+-------------+----------------+------------+--------------+----+-------------+---------------+----------------+-----------------+-----------+------------+----------------+-----------------+-----------

# 2.A
The 5 most frequent visitors (NAMELAST, NAMEFIRST, NAMEMID) to the
White House.

In [120]:
spark = SparkSession.builder.appName("WhiteHouseTopVisitors").getOrCreate()
visitors_df = visitors_df.na.fill(" ", subset=["NAMEMID"])
visitors_df = visitors_df.select(visitors_df.NAMELAST, visitors_df.NAMEFIRST, visitors_df.NAMEMID)
visitor_counts = visitors_df.groupBy("NAMELAST","NAMEFIRST","NAMEMID").count()
top_visitors = visitor_counts.orderBy(visitor_counts["count"].desc()).limit(5)
top_visitors.show()

+--------+---------+-------+-----+
|NAMELAST|NAMEFIRST|NAMEMID|count|
+--------+---------+-------+-----+
|  DEMEKE|    MARIA|       |   41|
| NEUFELD|     ADAM|       |   36|
|     KIM|      HUN|      S|   30|
|   MILBY|    SARAH|       |   25|
|  WIDGER|      ANN|       |   25|
+--------+---------+-------+-----+



# 2.B
The 5 most frequently visited people (VISITEE_NAMELAST, VISITEE_NAMEFIRST)
in the White House

In [121]:
visitors_df = visitors_df.withColumnRenamed("NAMELAST", 'VISITEE_NAMELAST')
visitors_df = visitors_df.withColumnRenamed("NAMEFIRST",'VISITEE_NAMEFIRST')

visitors_df = visitors_df.select(visitors_df.VISITEE_NAMELAST, visitors_df.VISITEE_NAMEFIRST)
visitor_counts = visitors_df.groupBy("VISITEE_NAMELAST","VISITEE_NAMEFIRST").count()
top_visitors = visitor_counts.orderBy(visitor_counts["count"].desc()).limit(5)
top_visitors.show()


+----------------+-----------------+-----+
|VISITEE_NAMELAST|VISITEE_NAMEFIRST|count|
+----------------+-----------------+-----+
|           MIRZA|             ASMA|   48|
|         NEUFELD|             ADAM|   43|
|          DEMEKE|            MARIA|   43|
|       CHILDRESS|             MARK|   39|
|         LAMBREW|           JEANNE|   36|
+----------------+-----------------+-----+



# 3.B
The 5 most frequent visitor-visitee combinations

In [122]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc
spark = SparkSession.builder.appName("WhiteHouseVisitorVisitee").getOrCreate()
combination = df.select("NAMELAST", "NAMEMID", "NAMEFIRST", "VISITEE_NAMELAST", "VISITEE_NAMEFIRST")\
.groupBy("NAMELAST", "NAMEMID", "NAMEFIRST","VISITEE_NAMELAST","VISITEE_NAMEFIRST")
combination.count().orderBy("count", ascending=False).show(5)

+--------+-------+---------+----------------+-----------------+-----+
|NAMELAST|NAMEMID|NAMEFIRST|VISITEE_NAMELAST|VISITEE_NAMEFIRST|count|
+--------+-------+---------+----------------+-----------------+-----+
|     KIM|      S|      HUN|          KUNDRA|            VIVEK|   22|
|  DEMEKE|   NULL|    MARIA|          BUSSOW|             MARK|   21|
|   YODER|   NULL|     RYAN|          SAHENE|              JR.|   19|
|  HARDIE|      D|     IRIS|          KUNDRA|            VIVEK|   18|
|  DEMEKE|   NULL|    MARIA|        WILLIAMS|            DEBRA|   17|
+--------+-------+---------+----------------+-----------------+-----+
only showing top 5 rows

